In [ ]:
import os
import time
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F



# Como fica o tamanho da dimensão para a operação de Conv2D



$$ out\_dim = \lceil\frac{in\_dim - kernel\_size + 1 + 2*padding}{stride}\rceil $$ 

In [33]:
#convt = nn.ConvTranspose2d(in_channels=nz, out_channels=ngf * 8, kernel_size=(4,4), stride=stride, padding=1, bias=False)


In [38]:
### stride = 1
in_dim = 12
img_size = 32 
for i in range(img_size + 5):
    entrada1 = torch.randn(1,in_dim, img_size, img_size)

    entrada1 = Variable(entrada1)
    convt = nn.Conv2d(in_channels=in_dim, out_channels=in_dim*2, kernel_size=(3,3), stride = i+1, padding = 2)
    

    entrada2 = convt(entrada1)
    #print('Para padding = ',i, 'A dimensao gerada é = ',entrada2.size())
    print('Para stride = ',i+1, 'A dimensao gerada é = ',entrada2.size())
    #plt.imshow(entrada2.data.numpy()[0,0,:,:],cmap='gray')

Para stride =  1 A dimensao gerada é =  torch.Size([1, 24, 34, 34])
Para stride =  2 A dimensao gerada é =  torch.Size([1, 24, 17, 17])
Para stride =  3 A dimensao gerada é =  torch.Size([1, 24, 12, 12])
Para stride =  4 A dimensao gerada é =  torch.Size([1, 24, 9, 9])
Para stride =  5 A dimensao gerada é =  torch.Size([1, 24, 7, 7])
Para stride =  6 A dimensao gerada é =  torch.Size([1, 24, 6, 6])
Para stride =  7 A dimensao gerada é =  torch.Size([1, 24, 5, 5])
Para stride =  8 A dimensao gerada é =  torch.Size([1, 24, 5, 5])
Para stride =  9 A dimensao gerada é =  torch.Size([1, 24, 4, 4])
Para stride =  10 A dimensao gerada é =  torch.Size([1, 24, 4, 4])
Para stride =  11 A dimensao gerada é =  torch.Size([1, 24, 4, 4])
Para stride =  12 A dimensao gerada é =  torch.Size([1, 24, 3, 3])
Para stride =  13 A dimensao gerada é =  torch.Size([1, 24, 3, 3])
Para stride =  14 A dimensao gerada é =  torch.Size([1, 24, 3, 3])
Para stride =  15 A dimensao gerada é =  torch.Size([1, 24, 3, 3]

## Rede D do Lotufo

In [ ]:
class _netD_DCGAN_MNIST(nn.Module):
    def __init__(self, nc, ndf=64):
        super().__init__()
        self.main = nn.Sequential(
            # input is (nc) x 28 x 28
            nn.Conv2d(nc, ndf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 14 x 14
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*2) x 7 x 7
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

Na hora de fazer a codificação da informação, por redes convolucionais, é interessante jogar a imagem para o teto da metade $\lceil \frac{n}{2} \rceil$ e usar stride um pouco maior e padding no ínicio (para guardar mais da informação da imagem).

## Nossa rede D

Note que não dimnuímos o padding e o stride do meio pra frente da rede, talvez para fazer a
imagem bater direitinho pois ela tinha tamanho de 64.

In [ ]:
class _netD_DCGAN(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf, n_classes):
        super(_netD_DCGAN, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Conv2d(in_channels = nc, out_channels = ndf, kernel_size=4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels = ndf, out_channels = ndf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(in_channels = ndf*2, out_channels = ndf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(in_channels = ndf*4, out_channels = ndf*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ndf * 8)
        
        #self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=n_classes+1,kernel_size=4,stride=1,padding=0,bias=False)
        self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=1,kernel_size=4,stride=1,padding=0,bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, inplace=True)
        x = F.leaky_relu(self.batch2(self.conv2(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch3(self.conv3(x)), 0.2, inplace=True)
        x = F.leaky_relu(self.batch4(self.conv4(x)), 0.2, inplace=True)
        x = self.final_conv(x)
        x = F.sigmoid(x)
        
        #x = x.view(-1, 1).squeeze(1)

        return(x)

# Como fica o tamanho da dimensão para a operação de Conv2D


$$ out\_dim = stride*in\_dim +kernel\_size -stride -2*padding $$

Até a dimensão mínima que  é (1,1)

In [36]:
stride = 1
in_dim = 100
img_size = 32 
for i in range(img_size + 5):
    entrada1 = torch.randn(1,in_dim, 10, 10)

    entrada1 = Variable(entrada1)
    
    convt = nn.ConvTranspose2d(in_channels=in_dim, out_channels= in_dim * 8, kernel_size=(4,4), stride=stride, padding=i,bias=False)


    entrada2 = convt(entrada1)
    print('Para padding = ',i, 'A dimensao gerada é = ',entrada2.size())
    #print('Para stride = ',i+1, 'A dimensao gerada é = ',entrada2.size())
    #plt.imshow(entrada2.data.numpy()[0,0,:,:],cmap='gray')

Para padding =  0 A dimensao gerada é =  torch.Size([1, 800, 13, 13])
Para padding =  1 A dimensao gerada é =  torch.Size([1, 800, 11, 11])
Para padding =  2 A dimensao gerada é =  torch.Size([1, 800, 9, 9])
Para padding =  3 A dimensao gerada é =  torch.Size([1, 800, 7, 7])
Para padding =  4 A dimensao gerada é =  torch.Size([1, 800, 5, 5])
Para padding =  5 A dimensao gerada é =  torch.Size([1, 800, 3, 3])
Para padding =  6 A dimensao gerada é =  torch.Size([1, 800, 1, 1])


RuntimeError: Given input size: (100 x 10 x 10). Calculated output size: (800 x -1 x -1). Output size is too small at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/THNN/generic/SpatialFullDilatedConvolution.c:107

## Rede G do Lotufo

In [ ]:
class _netG_DCGAN_MNIST(nn.Module):
    def __init__(self, nz, nc, ngf=64):
        super().__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            
            # state size. (ngf*4) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            
            # state size. (ngf*2) x 7 x 7
            nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            
            # state size. (ngf) x 14 x 14
            nn.ConvTranspose2d(ngf, nc, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
            
            # state size. (nc) x 28 x 28
        )

    def forward(self, input):
        output = self.main(input)
        return output

Não sei direito como funciona a convolução transposta, mas parece que ele apenas fez o caminho inverso da rede Conv2d

### Note que a conv2 tem parametros diferentes da rede D

In [40]:
class _netG_DCGAN(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf):
        super(_netG_DCGAN, self).__init__()
        self.ngpu = ngpu
        self.convt1 = nn.ConvTranspose2d(in_channels=nz, out_channels=ngf * 8, kernel_size=4, stride=1, padding=0, bias=False)
        self.batch1 = nn.BatchNorm2d(ngf*8)
        self.convt2 = nn.ConvTranspose2d(in_channels=ngf * 8, out_channels=ngf * 4, kernel_size=4, stride=1, padding=0, bias=False)
        self.batch2 = nn.BatchNorm2d(ngf*4)
        self.convt3 = nn.ConvTranspose2d(in_channels=ngf * 4, out_channels=ngf * 2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ngf*2)
        self.convt4 = nn.ConvTranspose2d(in_channels=ngf*2, out_channels=ngf, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ngf)
        
        self.final_convt = nn.ConvTranspose2d(in_channels=ngf, out_channels=nc, kernel_size=4, stride=2, padding=1, bias=False)
        
    def forward(self, x):
        print('1', x.size())
        x = F.leaky_relu(self.batch1(self.convt1(x)), 0.2, inplace=True)
        print('2', x.size())
        
        x = F.leaky_relu(self.batch2(self.convt2(x)), 0.2, inplace=True)
        print('3', x.size())
        
        x = F.leaky_relu(self.batch3(self.convt3(x)), 0.2, inplace=True)
        print('4', x.size())
        
        x = F.leaky_relu(self.batch4(self.convt4(x)), 0.2, inplace=True)
        print('5', x.size())
        
        x = self.final_convt(x)
        print('6', x.size())
        
        
        x = F.tanh(x)
        print('7', x.size())
        
        return (x)

# Resultado do teste da rede do Lotufo com LeakyRELU no lugar das RELU's e a rede igual a dele

### RELU

![](Relu_MNIST.png)


In [42]:
!ls

datasets		     naive_DCGAN.ipynb			README.MD
leaky.png		     note_teórico.ipynb			Relu_MNIST.png
naive_DCGAN_all_leaky.ipynb  outputdir_train_classifier_teste2


## LEAKY RELU

![](leaky.png)
